# More topics on Melbourne House Prices

This notebeook is a continuation of the notebook `house-pricing`. The idea is to go deep i nnew topics in ML.

First

## MISSING VALUES

Missing values are the most common data issue that you will be able to find in almost every data set. The best thing is to be prepared to face this challenge. 

Most of the machine learning libraries (like sci-kit learn) cannot deal with missing values. These raise an error if a model is built using data with missing values. To avoid these, there are some strategies.
 
**1. Drop columns with missing values**

This is the mos simple option but not the most recommended as it has to be clear that the column that is removed does not add anything to the model.

**2. Filling the missing values (Imputation)**

**Filling** the missing values with some other value. An example of a filling value is the _mean_ but in case you have [skewed](https://en.wikipedia.org/wiki/Skewness) data you might want to use the median, or even the mode. 

This is probably the best method that can be used. This depends on that the value that is added really makes sense as a value for the numerical values that are in the range of the column. 

**3. Extension of filling**

Filling is the standard approach, and it usually works well. There might be disadvantages as the value that is used to fill in the missing might not be completely accurate, or the rows with the missig values can be unique in some other way. In that case, the model would make better prediction considering which values were originlly missing. For that purpose, for each column with a missing value, a new column is added `"original_name"_was_missing` that has as values `True`if the value in the `original_name` was missing and `False` otherwise. This trick can help in some cases but is not always the case.

Now, let us test those three in the following.

In [1]:
# import all the necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

df = pd.read_csv('data/Melbourne_housing_FULL.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [14]:
#select the main variable
y = df.Price
y = y.fillna(y.median())

X = df.drop(['Price'], axis = 1)
#For simplicity select only the numerical variables
X = X.select_dtypes(exclude = ['object'])

# Divide the dataset
X_train, X_valid, y_train, y_valid= train_test_split(X, y, 
                                                     train_size = 0.8,
                                                     test_size = 0.2)

#function to compare the different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators =10, random_state = 100)
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    return mean_absolute_error(y_valid, pred)


In [15]:
# Case 1. drop the columns that contain missing values
cols_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(cols_missing, axis =1)
reduced_X_valid = X_valid.drop(cols_missing, axis = 1)

print('MAE for case1: Dropping columns with missng values : {}'.format(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [16]:
#Case 2: Imputer
from sklearn.impute import SimpleImputer

## Imputation
imputer = SimpleImputer()

imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

## gave back the original names to the columns

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print('MAE case 2: Filling/Imputation is:{}'.format(score_dataset(imputed_X_train,imputed_X_valid,y_train, y_valid)))

MAE case 2: Filling/Imputation is:256887.0926495192


There is already a considerable reduction in the MAE in between the first two approaches.

In [17]:
#Case three, Extension of filling
# make copies of the X sets
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

#make new column indicating what will be imputed
for col in cols_missing:
    X_train_plus[col+'_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col+'_was_missing'] = X_valid_plus[col].isnull()

#Generate the imputer and change the values
imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(imputer.transform(X_valid_plus))

# rename the columns of the imputer with the previous names
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print('MAE case 3: Imputer extended: {}'.format(score_dataset(imputed_X_train_plus,imputed_X_valid_plus,y_train, y_valid)))


MAE case 3: Imputer extended: 258894.00610422814


As seen in the last result, adding that extra layer of sophistication is not reflected in the overall model performance. 

## Categorical Data

A categorical variable takes only a limited number of values. Think for example surveys in which you are asked your mood and the possible answers are 'happy','normal', 'sad'. These are categories. 

How to use the data? 
**1. Drop Categorical variables**
This is again the easiest approach. Remove those columns that contain categorical values.
**2. Ordinal Encoding**
Assign a unique value to each of the categories. In this approach, it is assumed that there is an ordering of the variables. In the case, 'happy'>'normal'>'sad'
The assumtption makes sense in this example but this might not be the case all the time. These variables as referred as 'ordinal variables'
**3. One-hot Encoding**
One-hot encoding creates new columns indicating the presence (or absence) of each of the possible values in the original data. Each category is represented by a column that has the following form. For variable X and cateogry i, the variable $X_i$ takes the follwing values for the observation j
$$ X_i =\begin{cases}1 &\text{if }X(j) = i\\ 0& \text{otherwise} \end{cases}$$

Therefore, at the end, the new vectors contain the information that it was contained in the original categorical variable. 

In this approach, there is no order assumed, therefore this is a good approach when there is not a clear order in the categories. These are 'nominal variables'. One drawback from this approach is that it does not perform well if the categorical variable has too many categories where too many can be as small as 15 categories.

In [20]:
df

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.80140,144.99580,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.80790,144.99340,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.81140,145.01160,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.80930,144.99440,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0
